In [1]:
import pandas as pd
import numpy as np
import os
import subprocess
import datetime
from pathlib import Path
from screed import ScreedDB

In [9]:
metadata = pd.read_csv("../data/metadata_2021_04_08.tsv", sep="\t", parse_dates = ["Collection date", "Submission date"])
print(metadata.columns)
metadata = metadata[["Accession ID", \
                   "Collection date", \
                   "Submission date", \
                   "Location", \
                   "Additional location information", \
                   "Sequence length", \
                   "Host", \
                   "AA Substitutions", \
                   "Is reference?", \
                   "Pango lineage"]]
def get_nth_slash(row, n):
    try:
        return row.split("/")[n].strip()
    except:
        return np.nan
    
for i in range(4):
    metadata[f"Location_{i}"] = metadata["Location"].apply(lambda row: get_nth_slash(row, i))
metadata = metadata.rename(columns={"Location_1":"country", "Location_2":"state"})

Index(['Virus name', 'Type', 'Accession ID', 'Collection date', 'Location',
       'Additional location information', 'Sequence length', 'Host',
       'Patient age', 'Gender', 'Clade', 'Pango lineage', 'Pangolin version',
       'Variant', 'AA Substitutions', 'Submission date', 'Is reference?',
       'Is complete?', 'Is high coverage?', 'Is low coverage?', 'N-Content',
       'GC-Content'],
      dtype='object')


In [12]:
metadata[metadata.country == "USA"]["Pango lineage"].value_counts().head(50)

B.1.2        63442
B.1          30444
B.1.429      16400
B.1.1.7      15248
B.1.596       8439
B.1.243       8077
B.1.427       6871
B.1.234       4681
B.1.1.519     4359
B.1.526       4187
B.1.1         4170
A.1           2518
B.1.595       2421
B.1.240       2417
B.1.311       2170
B.1.369       2003
B.1.526.1     1851
B.1.1.434     1773
B.1.1.222     1648
B.1.517       1524
B.1.526.2     1375
B.1.564       1272
B.1.400       1224
B.1.577       1177
B.1.139       1161
B.1.1.291     1130
B.1.582        986
B.1.232        954
A              954
B.1.575        914
B.1.396        870
B.1.426        867
B.1.561        861
B.1.588        856
B.1.206        839
B.1.375        831
B.1.409        827
B.1.349        822
B.1.509        810
B.1.609        796
B              776
B.1.371        752
P.2            743
B.1.565        662
B.1.320        650
B.1.1.316      649
R.1            634
B.1.241        604
B.1.590        582
B.1.361        577
Name: Pango lineage, dtype: int64

In [3]:
def prep_gengraph_input_output(date, end_date, seq_names):
    date = date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")
    
    output_path = Path("/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston_output")
    run_path = output_path / f"{date}_{end_date}"
    
    if not run_path.exists():
        run_path.mkdir()
        
    seq_path = Path("/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston")
    seq_file_dest = run_path / f"{date}_{end_date}_seq_input.txt"
    with open(seq_file_dest, "w") as dest:
        dest.write("seq_name\taln_name\tseq_path\tannotation_path\n")
        for ix, seq in enumerate(seq_names):
            dest.write(f"{seq}\tseq_{ix}\t{seq_path / seq}.fasta\n")
            
    return run_path, seq_file_dest, f"{date}_{end_date}"

def run_pipeline(date, end_date, seq_names, exp_name, location="Houston"):
    location = location.replace(" ", "_")
    base_path = Path("/scratch/jho5ze/bionets/covid/")
    uncert_file = "uncert.csv"
    
    date = date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")
    
    output_path = Path("/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/output")
    run_path = output_path / location / exp_name / f"{date}_{end_date}"
    uncert_file = run_path / "uncert.csv"
    
    
    if not run_path.exists():
        run_path.mkdir(parents=True)
        
    seq_path = Path("/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston")
    
    command = f"sbatch {base_path / 'scripts/run_pipeline.sbatch'} {run_path} {uncert_file}".split()
#     command += [f"{seq_path / seq}.fasta" for seq in seq_names]
    command += [f"{seq}" for seq in seq_names]
    return command


def run_jaccard_pipeline(date, end_date, seq_names, exp_name, location="Houston"):
    location = location.replace(" ", "_")
    base_path = Path("/scratch/jho5ze/bionets/covid/")
    uncert_file = "uncert.csv"
    
    date = date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")
    
    output_path = Path("/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/output")
    run_path = output_path / location / exp_name / f"{date}_{end_date}"
    uncert_file = run_path / "jaccard_uncert.csv"
    
    
    if not run_path.exists():
        run_path.mkdir(parents=True)
        
    seq_path = Path("/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston")
    
    command = f"sbatch {base_path / 'scripts/run_jaccard_pipeline.sbatch'} {location} {date} {end_date} {uncert_file}".split()
#     command += [f"{seq_path / seq}.fasta" for seq in seq_names]
    command += [f"{seq}" for seq in seq_names]
    return command
#     subprocess.run(command)
#     with open(seq_file_dest, "w") as dest:
#         dest.write("seq_name\taln_name\tseq_path\tannotation_path\n")
#         for ix, seq in enumerate(seq_names):
#             dest.write(f"{seq}\tseq_{ix}\t{seq_path / seq}.fasta\n")
            
#     return run_path, seq_file_dest, f"{date}_{end_date}"

In [7]:
timeout = """Houston 2020-06-03 2020-06-09
Houston 2020-06-10 2020-06-16
Houston 2020-06-17 2020-06-23
Houston 2020-06-24 2020-06-30
Houston 2020-07-01 2020-07-07
Houston 2020-07-08 2020-07-14
Houston 2020-08-12 2020-08-18
Houston 2020-08-19 2020-08-25
Houston 2020-08-26 2020-09-01
Houston 2020-09-23 2020-09-29
Houston 2020-09-30 2020-10-06
Houston 2020-10-07 2020-10-13
Houston 2020-10-14 2020-10-20
Houston 2020-10-21 2020-10-27
Houston 2020-10-28 2020-11-03
Houston 2020-11-04 2020-11-10
Houston 2020-11-11 2020-11-17
Houston 2020-11-18 2020-11-24
Houston 2020-11-25 2020-12-01
Houston 2020-12-23 2020-12-29
Houston 2020-12-30 2021-01-05
Houston 2021-01-06 2021-01-12
Houston 2021-01-13 2021-01-19
Houston 2021-01-20 2021-01-26
Houston 2021-01-27 2021-02-02
Houston 2021-02-03 2021-02-09
Houston 2021-02-10 2021-02-16
Houston 2021-02-17 2021-02-23
Houston 2021-02-24 2021-03-02
Houston 2021-03-03 2021-03-09
New_York_City 2021-02-01 2021-02-07
New_York_City 2021-02-08 2021-02-14
New_York_City 2021-02-15 2021-02-21
New_York_City 2021-02-22 2021-02-28
New_York_City 2021-03-01 2021-03-07
New_York_City 2021-03-08 2021-03-14
New_York_City 2021-03-15 2021-03-21
New_York_City 2021-03-22 2021-03-28
New_York_City 2021-03-29 2021-04-04
San_Diego 2021-01-16 2021-01-22
Santa_Clara_County 2020-12-02 2020-12-08
Yakima_County 2020-05-26 2020-06-01
Yakima_County 2020-06-02 2020-06-08""".split("\n")
# timeout = "New_York_City 2021-02-22_2021-02-28"
# timeout

In [24]:
data_path = Path("/scratch/jho5ze/bionets/covid/data")

seqs = pd.read_csv(data_path / "houston_metadata.csv", header=None, parse_dates=[1])
seqs.columns = ["seq", "date"]

date_window = 7
date_step_size = 7

for date in pd.date_range(seqs.date.min(), seqs.date.max(), freq=f"{date_step_size}D"):
    end_date = date + np.timedelta64(date_window - 1, 'D')
    sub_seqs = seqs[(seqs.date >= date) & (seqs.date <= end_date)].seq
#     output_directory, seq_file, out_file = prep_gengraph_input_output(date, end_date, sub_seqs)
    experiment_name = f"window_{date_window}_step_{date_step_size}"
    stdate = date.strftime("%Y-%m-%d")
    stend_date = end_date.strftime("%Y-%m-%d")
    
#     if f"{stdate}_{stend_date}" in timeout:
#     command = run_pipeline(date, end_date, sub_seqs, experiment_name)
#     subprocess.run(command)
#     print(f"{stdate}_{stend_date}")
#     print("\t", len(sub_seqs))
#     break
#     print(" ".join(command))
#     break
        
#         command = f"sbatch {base_path / 'scripts/run_gengraph.sbatch'} {output_directory} {seq_file} {out_file}"
#         subprocess.run(command.split())
#     break


In [66]:
metadata[metadata.country == "USA"].Location_3.shape #value_counts()[:10]

(246062,)

In [74]:
metadata[metadata.country == "USA"].Location_3.isna().sum()

139992

In [48]:
metadata[metadata["Accession ID"] == "EPI_ISL_1303700"]

,Accession ID,Collection date,Submission date,Location,Additional location information,Sequence length,Host,AA Substitutions,Is reference?,Location_0,country,state,Location_3
830371,EPI_ISL_1303700,2021-02-24,2021-03-21,North America / USA / Texas / Houston,NaN,29859,Human,"(NSP6_G107del,N_S194L,NSP6_S106del,Spike_E484K...",NaN,North America,USA,Texas,Houston


In [4]:
msadb = ScreedDB("../data/msa_0408/usa_msa_0408.fasta")
# msadb_keys = msadb.keys()

In [5]:
top_k = 10
top_k_sequenced_us = metadata[metadata.country == "USA"].Location_3.value_counts()[:top_k].index
top_state_county = []
for i in top_k_sequenced_us:
    top_state = metadata[(metadata.country == "USA") & (metadata.Location_3 == i)].state.value_counts().index[0]
#     print(i, ":", top_state)
#     print()
    top_state_county.append((top_state, i))

In [6]:
top_state_county

[('Texas', 'Houston'),
 ('New York', 'New York City'),
 ('California', 'San Diego'),
 ('California', 'Santa Clara County'),
 ('Wisconsin', 'Dane County'),
 ('California', 'Alameda County'),
 ('Washington', 'Yakima County'),
 ('Washington', 'King County'),
 ('California', 'Orange County'),
 ('California', 'Los Angeles County')]

In [6]:
loc_date_samples = []

In [9]:
# location = "Houston"
"""
Need to redo Orange county (had a few NY sequences and a few others in there too)
"""
for state, location in top_state_county:
#     if location == "Houston":
#         continue
        
#     if location !="King County": continue 
        
    location_metadata = metadata[(metadata.country == "USA") & (metadata.state == state) & (metadata.Location_3 == location)]
    location_metadata = location_metadata[["Accession ID", "Collection date"]]
    location_metadata.columns = ["seq", "date"]
#     location_metadata.date = pd.to_datetime(location_metadata.date, errors="coerce")
    location_metadata = location_metadata.dropna()
    
    date_window = 7
    date_step_size = 7

    for date in pd.date_range(location_metadata.date.min(), location_metadata.date.max(), freq=f"{date_step_size}D"):
        end_date = date + np.timedelta64(date_window - 1, 'D')
        sub_seqs = location_metadata[(location_metadata.date >= date) & (location_metadata.date <= end_date)].seq
        sub_seqs = [s for s in sub_seqs if s in msadb]
    #     output_directory, seq_file, out_file = prep_gengraph_input_output(date, end_date, sub_seqs)
        experiment_name = f"window_{date_window}_step_{date_step_size}"
        stdate = date.strftime("%Y-%m-%d")
        stend_date = end_date.strftime("%Y-%m-%d")
#         if f"{stdate}_{stend_date}" != "2020-12-25_2020-12-31": continue
        
#         command = run_pipeline(date, end_date, sub_seqs, experiment_name, location)
        command = run_jaccard_pipeline(date, end_date, sub_seqs, experiment_name, location)
        
        if len(sub_seqs) > 0:
            if location.replace(" ", "_") + " " + f"{stdate} {stend_date}" in timeout:
#                 print(location, stdate, len(sub_seqs))
#                 break
                subprocess.run(command)
#             pass
#         else:
#         loc_date_samples.append((location, stdate, len(sub_seqs)))
        
#         print(command)
#         break
#     break
#         if len(sub_seqs) > 0:
#             subprocess.run(command)
#     break

In [11]:
import pickle

pickle.dump(loc_date_samples, open("loc_date_samples.pkl", "wb")) #.tofile("loc_date_samples.npy")